## Chapter 10. 케라스를 사용한 인공 신경망 소개

#### 10.1 생물학적 뉴런에서 인공 뉴런까지

#### 퍼셉트론

In [3]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron
import warnings; warnings.filterwarnings('ignore')

iris = load_iris()
X = iris.data[:, (2, 3)] # 꽃잎의 길이와 너비
y = (iris.target == 0).astype(np.int) # 부채붓꽃(Iris Setosa)인가?

per_clf = Perceptron()
per_clf.fit(X, y)

y_pred = per_clf.predict([[2, 0.5]])
y_pred

array([0])

#### 10.2 케라스로 다층 퍼셉트론 구현하기